In [1]:
import sys, pandas as pd
print("Python:", sys.version)
print("Executable:", sys.executable)
print("Pandas:", pd.__version__)


Python: 3.11.14 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 18:30:03) [MSC v.1929 64 bit (AMD64)]
Executable: c:\Users\zhaoyuan\Downloads\python\envs\takeaway\python.exe
Pandas: 2.3.3


In [2]:
import pandas as pd

RAW_PATH = "../data/raw/purchasing-commodity-data.csv"

df = pd.read_csv(
    RAW_PATH,
    nrows=100_000,
    low_memory=False
)

df.shape


(100000, 20)

In [3]:
cols = df.columns.tolist()
len(cols), cols[:20]
cols


['Fiscal Year',
 'Purchase Order',
 'Purchase Order Line',
 'Contract Number',
 'Purchasing Department',
 'Purchasing Department Title',
 'Post Date - Original',
 'Post Date - Current',
 'Commodity Code',
 'Commodity Title',
 'Vendor Name',
 'Vendor Street',
 'Vendor City',
 'Vendor State',
 'Vendor ZIP Code',
 'Vendor Contact',
 'Vendor Email',
 'Vendor Phone',
 'Encumbered Quantity',
 'Encumbered Amount']

In [4]:
profile = (
    pd.DataFrame({
        "dtype": df.dtypes.astype(str),
        "non_null": df.notnull().sum(),
        "null_ratio": 1 - df.notnull().mean()
    })
    .sort_values(["null_ratio", "non_null"], ascending=[True, False])
)

profile.head(25)
profile.tail(25)


,dtype,non_null,null_ratio
Fiscal Year,int64,100000,0.00000
Purchase Order,object,100000,0.00000
Purchase Order Line,int64,100000,0.00000
Purchasing Department Title,object,100000,0.00000
Post Date - Current,object,100000,0.00000
Commodity Title,int64,100000,0.00000
Vendor Name,object,100000,0.00000
Vendor Street,object,100000,0.00000
Vendor City,object,100000,0.00000
Vendor State,object,100000,0.00000


## ERP System Split Assumptions

To simulate a multi-ERP enterprise procurement landscape:

- ERP_A represents a legacy procurement system (pre-2019)
- ERP_B represents a modernized ERP system (2019 onwards)

The split is based on fiscal year and reflects differences in:
- Date fields
- Category granularity
- Organizational naming conventions
- Vendor master completeness


In [5]:
import pandas as pd

RAW_PATH = "../data/raw/purchasing-commodity-data.csv"
OUT_A = "../data/processed/erp_a_spend.csv"
OUT_B = "../data/processed/erp_b_spend.csv"


df = pd.read_csv(
    RAW_PATH,
    nrows=100_000,
    low_memory=False
)


# ERP_A: Legacy system (<= 2018)
erp_a = df[df["Fiscal Year"] <= 2018].copy()

# ERP_B: Modern system (>= 2019)
erp_b = df[df["Fiscal Year"] >= 2019].copy()




# ERP_A uses older / less descriptive fields
erp_a = erp_a.rename(columns={
    "Post Date - Original": "posting_date",
    "Commodity Code": "category_code",
    "Purchasing Department": "department_code",
    "Encumbered Amount": "amount"
})

erp_a["erp_system"] = "ERP_A"

# ERP_B uses modern naming & richer descriptions
erp_b = erp_b.rename(columns={
    "Post Date - Current": "posting_date",
    "Commodity Title": "category_name",
    "Purchasing Department Title": "department_name",
    "Encumbered Amount": "amount"
})

erp_b["erp_system"] = "ERP_B"

# Persist
erp_a.to_csv(OUT_A, index=False)
erp_b.to_csv(OUT_B, index=False)

len(erp_a), len(erp_b)


(100000, 0)

In [6]:
import os

RAW_PATH = "../data/raw/purchasing-commodity-data.csv"

print("exists:", os.path.exists(RAW_PATH))
print("size_mb:", round(os.path.getsize(RAW_PATH)/1024/1024, 2))
print("abs_path:", os.path.abspath(RAW_PATH))


exists: True
size_mb: 663.43
abs_path: c:\Users\zhaoyuan\Desktop\procurement_spend_analysis\data\raw\purchasing-commodity-data.csv


In [7]:
with open(RAW_PATH, "rb") as f:
    head = f.read(3000)

print(head[:300])  # 只打印前 300 字节


b'Fiscal Year,Purchase Order,Purchase Order Line,Contract Number,Purchasing Department,Purchasing Department Title,Post Date - Original,Post Date - Current,Commodity Code,Commodity Title,Vendor Name,Vendor Street,Vendor City,Vendor State,Vendor ZIP Code,Vendor Contact,Vendor Email,Vendor Phone,Encumbe'


In [8]:
with open(RAW_PATH, "r", encoding="utf-8", errors="replace") as f:
    for i in range(5):
        print(repr(f.readline()))


'Fiscal Year,Purchase Order,Purchase Order Line,Contract Number,Purchasing Department,Purchasing Department Title,Post Date - Original,Post Date - Current,Commodity Code,Commodity Title,Vendor Name,Vendor Street,Vendor City,Vendor State,Vendor ZIP Code,Vendor Contact,Vendor Email,Vendor Phone,Encumbered Quantity,Encumbered Amount\n'
'2006,DPPT06500258,001,,,DPT,,2005-12-15T00:00:00.000,12/15/2005,972016201300,SHOVEL,S & S SUPPLIES & SOLUTIONS,2700 MAXWELL WAY,FAIRFIELD,CA,94534,,,925-313-0361,2.00\n'
'2006,DPPT06501028,010,,,DPT,,2006-02-03T00:00:00.000,02/03/2006,97201620,HAND TOOLS,GRAINGER,750 BRANNAN ST,SAN FRANCISCO,CA,94103,,,415-864-4700,1.00\n'
'2006,DPPT06501363,005,,,DPT,,2006-05-03T00:00:00.000,05/03/2006,97201620,HAND TOOLS,HISCO,814-B 12TH ST,OAKLAND,CA,94607,STEVE JACOBSON,,510-776-2127,9.00\n'
'2006,DPPT06501908,002,,,DPT,,2006-04-20T00:00:00.000,04/20/2006,92156607,"GLOVES, WORK",S & S SUPPLIES & SOLUTIONS,2700 MAXWELL WAY,FAIRFIELD,CA,94534,,,925-313-0361,50.00\n'


In [9]:
import pandas as pd

df5 = pd.read_csv(RAW_PATH, nrows=5, sep=",", engine="python")
print(df5.shape)
print(df5.columns.tolist())
df5.head()


(5, 20)
['Fiscal Year', 'Purchase Order', 'Purchase Order Line', 'Contract Number', 'Purchasing Department', 'Purchasing Department Title', 'Post Date - Original', 'Post Date - Current', 'Commodity Code', 'Commodity Title', 'Vendor Name', 'Vendor Street', 'Vendor City', 'Vendor State', 'Vendor ZIP Code', 'Vendor Contact', 'Vendor Email', 'Vendor Phone', 'Encumbered Quantity', 'Encumbered Amount']


,Fiscal Year,Purchase Order,Purchase Order Line,Contract Number,Purchasing Department,Purchasing Department Title,Post Date - Original,Post Date - Current,Commodity Code,Commodity Title,Vendor Name,Vendor Street,Vendor City,Vendor State,Vendor ZIP Code,Vendor Contact,Vendor Email,Vendor Phone,Encumbered Quantity,Encumbered Amount
0,2006,DPPT06500258,1,NaN,NaN,DPT,NaN,2005-12-15T00:00:00.000,12/15/2005,972016201300,SHOVEL,S & S SUPPLIES & SOLUTIONS,2700 MAXWELL WAY,FAIRFIELD,CA,94534,NaN,NaN,925-313-0361,2.0
1,2006,DPPT06501028,10,NaN,NaN,DPT,NaN,2006-02-03T00:00:00.000,02/03/2006,97201620,HAND TOOLS,GRAINGER,750 BRANNAN ST,SAN FRANCISCO,CA,94103,NaN,NaN,415-864-4700,1.0
2,2006,DPPT06501363,5,NaN,NaN,DPT,NaN,2006-05-03T00:00:00.000,05/03/2006,97201620,HAND TOOLS,HISCO,814-B 12TH ST,OAKLAND,CA,94607,STEVE JACOBSON,NaN,510-776-2127,9.0
3,2006,DPPT06501908,2,NaN,NaN,DPT,NaN,2006-04-20T00:00:00.000,04/20/2006,92156607,"GLOVES, WORK",S & S SUPPLIES & SOLUTIONS,2700 MAXWELL WAY,FAIRFIELD,CA,94534,NaN,NaN,925-313-0361,50.0
4,2006,DPPT06501908,3,NaN,NaN,DPT,NaN,2006-04-20T00:00:00.000,04/20/2006,92156607,"GLOVES, WORK",S & S SUPPLIES & SOLUTIONS,2700 MAXWELL WAY,FAIRFIELD,CA,94534,NaN,NaN,925-313-0361,150.0


In [10]:
import pandas as pd

RAW_PATH = "../data/raw/purchasing-commodity-data.csv"

df = pd.read_csv(
    RAW_PATH,
    nrows=100_000,
    sep=",",
    engine="python",          # 更能容忍不规则行
    on_bad_lines="skip",      # 跳过字段数不一致的坏行
)

df.shape, df.columns.tolist()[:10]


((100000, 20),
 ['Fiscal Year',
  'Purchase Order',
  'Purchase Order Line',
  'Contract Number',
  'Purchasing Department',
  'Purchasing Department Title',
  'Post Date - Original',
  'Post Date - Current',
  'Commodity Code',
  'Commodity Title'])

In [11]:
bad_row_ratio = 1 - (len(df) / 100_000)
print("rows_loaded:", len(df))
print("approx_bad_row_ratio:", round(bad_row_ratio, 4))


rows_loaded: 100000
approx_bad_row_ratio: 0.0


In [12]:
OUT_A = "../data/processed/erp_a_spend.csv"
OUT_B = "../data/processed/erp_b_spend.csv"


df["Fiscal Year"] = pd.to_numeric(df["Fiscal Year"], errors="coerce")

erp_a = df[df["Fiscal Year"] <= 2018].copy()
erp_b = df[df["Fiscal Year"] >= 2019].copy()


erp_a = erp_a.rename(columns={
    "Post Date - Original": "posting_date",
    "Commodity Code": "category_code",
    "Purchasing Department": "department_code",
    "Encumbered Amount": "amount"
})
erp_a["erp_system"] = "ERP_A"

erp_b = erp_b.rename(columns={
    "Post Date - Current": "posting_date",
    "Commodity Title": "category_name",
    "Purchasing Department Title": "department_name",
    "Encumbered Amount": "amount"
})
erp_b["erp_system"] = "ERP_B"

erp_a.to_csv(OUT_A, index=False)
erp_b.to_csv(OUT_B, index=False)

erp_a.shape, erp_b.shape


((100000, 21), (0, 21))

In [13]:
df["Fiscal Year"].min(), df["Fiscal Year"].max(), df["Fiscal Year"].value_counts().head()


(1999,
 2007,
 Fiscal Year
 2007    84591
 2006    14584
 2005      640
 2004      156
 2003       22
 Name: count, dtype: int64)

In [14]:
import pandas as pd
import os

RAW_PATH = "../data/raw/purchasing-commodity-data.csv"
OUT_A = "../data/processed/erp_a_spend.csv"
OUT_B = "../data/processed/erp_b_spend.csv"


for p in [OUT_A, OUT_B]:
    if os.path.exists(p):
        os.remove(p)

chunksize = 200_000  

first_a = True
first_b = True

usecols = [
    "Fiscal Year","Purchase Order","Purchase Order Line","Contract Number",
    "Purchasing Department","Purchasing Department Title",
    "Post Date - Original","Post Date - Current",
    "Commodity Code","Commodity Title",
    "Vendor Name","Vendor Street","Vendor City","Vendor State","Vendor ZIP Code",
    "Vendor Contact","Vendor Email","Vendor Phone",
    "Encumbered Quantity","Encumbered Amount"
]

for chunk in pd.read_csv(
    RAW_PATH,
    usecols=usecols,
    chunksize=chunksize,
    sep=",",
    engine="python",
    on_bad_lines="skip"
):
    chunk["Fiscal Year"] = pd.to_numeric(chunk["Fiscal Year"], errors="coerce")

    a = chunk[chunk["Fiscal Year"] <= 2018].copy()
    b = chunk[chunk["Fiscal Year"] >= 2019].copy()

    
    if len(a) > 0:
        a = a.rename(columns={
            "Post Date - Original": "posting_date",
            "Commodity Code": "category_code",
            "Purchasing Department": "department_code",
            "Encumbered Amount": "amount"
        })
        a["erp_system"] = "ERP_A"
        a.to_csv(OUT_A, mode="a", header=first_a, index=False)
        first_a = False

    if len(b) > 0:
        b = b.rename(columns={
            "Post Date - Current": "posting_date",
            "Commodity Title": "category_name",
            "Purchasing Department Title": "department_name",
            "Encumbered Amount": "amount"
        })
        b["erp_system"] = "ERP_B"
        b.to_csv(OUT_B, mode="a", header=first_b, index=False)
        first_b = False

print("done")
print("erp_a_exists:", os.path.exists(OUT_A), "size_mb:", round(os.path.getsize(OUT_A)/1024/1024, 2) if os.path.exists(OUT_A) else 0)
print("erp_b_exists:", os.path.exists(OUT_B), "size_mb:", round(os.path.getsize(OUT_B)/1024/1024, 2) if os.path.exists(OUT_B) else 0)


done
erp_a_exists: True size_mb: 420.93
erp_b_exists: True size_mb: 259.43


## Target Harmonized Spend Schema (v1)

The harmonized spend layer aligns legacy and modern ERP systems into a unified analytical model.

Core fields:
- transaction_id (PO + line)
- fiscal_year
- posting_date
- vendor_name
- spend_amount
- category
- department
- erp_system

This layer is designed to support enterprise-wide spend analytics and BI reporting.


In [15]:
import pandas as pd

ERP_A_PATH = "../data/processed/erp_a_spend.csv"
ERP_B_PATH = "../data/processed/erp_b_spend.csv"

a = pd.read_csv(ERP_A_PATH, nrows=50_000, low_memory=False)
b = pd.read_csv(ERP_B_PATH, nrows=50_000, low_memory=False)

a.shape, b.shape


((50000, 21), (50000, 21))

In [16]:
def harmonize_a(df):
    return pd.DataFrame({
        "transaction_id": df["Purchase Order"].astype(str) + "-" + df["Purchase Order Line"].astype(str),
        "fiscal_year": df["Fiscal Year"],
        "posting_date": df["posting_date"],
        "vendor_name": df["Vendor Name"],
        "spend_amount": df["amount"],
        "category": df.get("category_code"),
        "department": df.get("department_code"),
        "erp_system": df["erp_system"]
    })

def harmonize_b(df):
    return pd.DataFrame({
        "transaction_id": df["Purchase Order"].astype(str) + "-" + df["Purchase Order Line"].astype(str),
        "fiscal_year": df["Fiscal Year"],
        "posting_date": df["posting_date"],
        "vendor_name": df["Vendor Name"],
        "spend_amount": df["amount"],
        "category": df.get("category_name"),
        "department": df.get("department_name"),
        "erp_system": df["erp_system"]
    })

ha = harmonize_a(a)
hb = harmonize_b(b)

harmonized = pd.concat([ha, hb], ignore_index=True)
harmonized.shape


(100000, 8)

In [17]:
harmonized["posting_date_parsed"] = pd.to_datetime(
    harmonized["posting_date"],
    errors="coerce",
    infer_datetime_format=True
)

harmonized[["posting_date", "posting_date_parsed"]].head(10)


C:\Users\zhaoyuan\AppData\Local\Temp\ipykernel_27752\2684871293.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  harmonized["posting_date_parsed"] = pd.to_datetime(


,posting_date,posting_date_parsed
0,NaN,NaT
1,NaN,NaT
2,NaN,NaT
3,NaN,NaT
4,NaN,NaT
5,NaN,NaT
6,NaN,NaT
7,NaN,NaT
8,NaN,NaT
9,NaN,NaT


In [18]:
harmonized["category_normalized"] = harmonized["category"].astype(str).str.strip().str.upper()

harmonized[["category", "category_normalized"]].head(10)


,category,category_normalized
0,12/15/2005,12/15/2005
1,02/03/2006,02/03/2006
2,05/03/2006,05/03/2006
3,04/20/2006,04/20/2006
4,04/20/2006,04/20/2006
5,04/20/2006,04/20/2006
6,06/22/2006,06/22/2006
7,06/22/2006,06/22/2006
8,06/22/2006,06/22/2006
9,06/28/2006,06/28/2006


In [19]:
harmonized["department_normalized"] = harmonized["department"].astype(str).str.strip()


## Core Procurement Spend KPIs

The harmonized spend layer supports the following core procurement KPIs:

1. Total Spend
2. Spend Trend (YoY)
3. Top Vendors by Spend
4. Vendor Concentration Ratio
5. Spend by Category
6. Spend by Department
7. ERP System Spend Mix


In [20]:
total_spend = harmonized["spend_amount"].sum()
total_spend


np.float64(15702039659.460001)

In [21]:
spend_by_year = (
    harmonized
    .groupby("fiscal_year", as_index=False)["spend_amount"]
    .sum()
    .sort_values("fiscal_year")
)

spend_by_year


,fiscal_year,spend_amount
0,1999,0.000000e+00
1,2002,0.000000e+00
2,2003,0.000000e+00
3,2004,1.239000e+03
4,2005,1.299600e+04
5,2006,1.618017e+06
6,2007,2.361350e+06
7,2019,5.913622e+09
8,2020,9.213724e+09
9,2021,5.707000e+08


In [22]:
top_vendors = (
    harmonized
    .groupby("vendor_name", as_index=False)["spend_amount"]
    .sum()
    .sort_values("spend_amount", ascending=False)
    .head(10)
)

top_vendors


,vendor_name,spend_amount
2404,MIDWEST TAPE LLC,3.402706e+09
4031,US FOODS INC,3.220761e+09
2340,MCKESSON CORPORATION,3.140751e+09
2402,MIDWEST LIBRARY SERVICE,1.170733e+09
4021,UNIVAR SOLUTIONS USA INC,1.003844e+09
1652,GOLDEN GATE PETROLEUM,4.281968e+08
3972,TURNER CONSTRUCTION CO,3.217771e+08
1585,GCS ENVIRONMENTAL EQUIPMENT SERVICES INC,2.102830e+08
1924,IPS GROUP INC,1.180472e+08
3401,SKANSKA CONSTRUCTORS,1.120811e+08


In [23]:
vendor_concentration = (
    top_vendors["spend_amount"].sum() / total_spend
)

vendor_concentration


np.float64(0.8361449503217925)

In [24]:
spend_by_category = (
    harmonized
    .groupby("category_normalized", as_index=False)["spend_amount"]
    .sum()
    .sort_values("spend_amount", ascending=False)
    .head(10)
)

spend_by_category


,category_normalized,spend_amount
2333,PUBLICATIONS AUDIOVISUAL MATERIALS BOOKS TEXT...,4.574625e+09
1716,FOODS: STAPLE GROCERY AND GROCER'S MISCELLANEO...,3.247102e+09
1851,HOSPITAL AND SURGICAL EQUIPMENT INSTRUMENTS AN...,3.183860e+09
1323,CHEMICALS AND SOLVENTS COMMERCIAL (IN BULK),1.006317e+09
1408,CONSTRUCTION SERVICES GENERAL (INCL. MAINTENAN...,6.958952e+08
1731,FUEL OIL GREASE AND LUBRICANTS,4.284890e+08
1859,HUMAN SERVICES,2.519126e+08
1117,AUTOMOTIVE AND TRAILER EQUIPMENT AND PARTS,2.209621e+08
1835,HEALTH CARE SERVICES (NOT OTHERWISE CLASSIFIED),1.993108e+08
1796,GRANTS COMMUNITY SERVICE PROGRAMS,1.774048e+08


In [25]:
spend_by_department = (
    harmonized
    .groupby("department_normalized", as_index=False)["spend_amount"]
    .sum()
    .sort_values("spend_amount", ascending=False)
    .head(10)
)

spend_by_department


,department_normalized,spend_amount
3176,nan,1.570075e+10
2762,BPSF00003103,6.405000e+05
2731,BPSF00003026,1.320000e+05
3074,BPUC06500020,1.014000e+05
2704,BPSF00002026,7.000000e+04
2708,BPSF00002034,5.000000e+04
2711,BPSF00002048,2.901300e+04
2765,BPSF00003109,2.713609e+04
2783,BPSF00003217,2.332960e+04
2786,BPSF00003223,1.465000e+04


In [26]:
spend_by_erp = (
    harmonized
    .groupby("erp_system", as_index=False)["spend_amount"]
    .sum()
)

spend_by_erp


,erp_system,spend_amount
0,ERP_A,3.993602e+06
1,ERP_B,1.569805e+10


In [27]:
harmonized.to_csv("../data/processed/harmonized_spend_sample.csv", index=False)


In [28]:
import pandas as pd

p = "../data/processed/harmonized_spend_sample.csv"
df_check = pd.read_csv(p, nrows=20)

print(df_check.columns)
print(df_check[["erp_system","posting_date"]].head(10))
print("posting_date empty ratio:", (df_check["posting_date"].isna() | (df_check["posting_date"].astype(str).str.strip()=="")).mean())


Index(['transaction_id', 'fiscal_year', 'posting_date', 'vendor_name',
       'spend_amount', 'category', 'department', 'erp_system',
       'posting_date_parsed', 'category_normalized', 'department_normalized'],
      dtype='object')
  erp_system  posting_date
0      ERP_A           NaN
1      ERP_A           NaN
2      ERP_A           NaN
3      ERP_A           NaN
4      ERP_A           NaN
5      ERP_A           NaN
6      ERP_A           NaN
7      ERP_A           NaN
8      ERP_A           NaN
9      ERP_A           NaN
posting_date empty ratio: 1.0
